In [30]:
#exc1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
from datetime import timedelta
from openpyxl.utils import get_column_letter

In [31]:
#exc2
#Sheet nov2019_Mg_PS

#Step1: Load the Data for the month
df_nov2019_Mg_PS =  pd.read_excel(r'C:\Users\v.t.flores\Documents\Analysis_November2019_MSdataCLEAN.xlsx', sheet_name='NOV19Datetime_Mg_PS', index_col=False)

#Step2: Parse the DATETIME column
df_nov2019_Mg_PS['DATETIME'] = df_nov2019_Mg_PS['DATETIME'].apply(lambda x: parse(str(x)))

#Step3: Set the DATETIME column as the index
df_nov2019_Mg_PS.set_index('DATETIME', inplace=True)

#exc3
#Sheet 106FT02A_spcl

#Step1: Load the Data for the month
df_106FT02A_spcl =  pd.read_excel(r'C:\Users\v.t.flores\Documents\Analysis_November2019_MSdataCLEAN.xlsx', sheet_name='106FT02A_spcl', index_col=False)

#Step2: Parse the DATETIME column
df_106FT02A_spcl['DATETIME'] = df_106FT02A_spcl['DATETIME'].apply(lambda x: parse(x))

#Step3: Set the DATETIME column as the index
df_106FT02A_spcl.set_index('DATETIME', inplace=True)

#exc4
#Sheet 106FT02B_spcl

#Step1: Load the Data for the month
df_106FT02B_spcl = pd.read_excel(r'C:\Users\v.t.flores\Documents\Analysis_November2019_MSdataCLEAN.xlsx', sheet_name='106FT02B_spcl', index_col=False)

#Step2: Parse the DATETIME column
df_106FT02B_spcl['DATETIME'] = df_106FT02B_spcl['DATETIME'].apply(lambda x: parse(x))

#Step3: Set the DATETIME column as the index
df_106FT02B_spcl.set_index('DATETIME', inplace=True)

#exc5
#Sheet From Pi
#Step1: Load the Data for the month
df_nov2019Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\Analysis_November2019_MSdataCLEAN.xlsx', sheet_name='From Pi', index_col=False)

#Drop the row[1]
df_nov2019Pi = df_nov2019Pi.drop(df_nov2019Pi.index[0])

#Step2: Parse the DATETIME column
df_nov2019Pi['DATETIME'] = df_nov2019Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_nov2019Pi.set_index('DATETIME', inplace=True)

#exc6
#This code merges the formatted data from the three sheets to on single dataframe named df_comb

df_com_Pi_FT02A = df_nov2019Pi.join(df_106FT02A_spcl, how='outer')
df_com_Mg_FT02B = df_nov2019_Mg_PS.join(df_106FT02B_spcl, how='outer')
df_comb = df_com_Pi_FT02A.join(df_com_Mg_FT02B, how='outer')


#exc7
#finding peaks FT for A_filtration time

df_comb['A_t_FEED'] = pd.to_numeric(df_comb['A_t_FEED'], errors='coerce')
a = np.diff(np.sign(np.diff(df_comb['A_t_FEED']))).nonzero()[0] + 1               # local min & max
b = (np.diff(np.sign(np.diff(df_comb['A_t_FEED']))) > 0).nonzero()[0] + 1         # local min
c = (np.diff(np.sign(np.diff(df_comb['A_t_FEED']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number


#finding peaks FT for A_N2 Dry time

df_comb['A_t_DRY'] = pd.to_numeric(df_comb['A_t_DRY'], errors='coerce')
d = np.diff(np.sign(np.diff(df_comb['A_t_DRY']))).nonzero()[0] + 1               # local min & max
e = (np.diff(np.sign(np.diff(df_comb['A_t_DRY']))) > 0).nonzero()[0] + 1         # local min
f = (np.diff(np.sign(np.diff(df_comb['A_t_DRY']))) < 0).nonzero()[0] + 1         # local max
# +1 due to the fact that diff reduces the original index number

#Use b as the starting index and f as the ending index for one full cylcle of Filtration to N2 Drying

#-----------------------------------------------------------------------------------------------------------------------

#This code here collects the index range for the each cycle
try:
    ft_cyc_lim = {}

    for i in range(0, len(b)):
        for j in range(0, len(f)):
            if f[j] > b[i] and f[j] < b[i+1]:
                if f[j+1]<b[i+1]:
                    cyc_start = b[i]
                    cyc_end = f[j+1]
                    ft_cyc_lim.setdefault(cyc_start,cyc_end)

except IndexError:
    print('\nDone')
#------------------------------------------------------------     

print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in sign
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in sign
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in sign
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in less
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in sign
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in sign
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages


Done
Done 2020-01-17 13:37:36.919951


In [32]:
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later


#code for getting the CWP for CakeWash1 and CakeWash2
##Working as of 1/15/2020
A_CWP_1new = list()
A_CWP_1new[:] = []

A_CWP_1_indexList = list()
A_CWP_1_indexList[:] = []

A_CWP_2new = list()
A_CWP_2new[:] = []

A_CWP_2_indexList = list()
A_CWP_2_indexList[:] = []

A_CWP_1_inner = list()
A_CWP_1_inner[:] = []

A_CWP_2_inner = list()
A_CWP_2_inner[:] = []

A_Pair_CWP_1_List = list()
A_Pair_CWP_2_List = list()

try:
    for i in range(0, len(keyslist)):
        A_CWPList_i = list()
        A_CWPList_i[:] = []
        d = list()
        d[:] = []
        
        A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
        d = (np.diff(np.sign(np.diff(df_comb['106FT02A_CWP'].iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
        A_CWP_for_sort = list()
        A_CWP_for_sort[:] = [] 
        
        A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        A_CWP_2_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
        
        for i in range(0, len(d)):
            CWP_val_shortlist = df_comb['106FT02A_CWP'].iloc[A_CWPList_i[d[i]]]
            A_CWP_for_sort.append(CWP_val_shortlist)
                    
        A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
        A_CWP_1_inner.append(A_CWP_1)
        
        A_CWP_2 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
        A_CWP_2_inner.append(A_CWP_2)

    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    A_Pair_CWP_2_List = list(zip(A_CWP_2_inner, A_CWP_2_indexList[0:]))
            
    
except IndexError:
    print('IndexError')

df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
df_A_Pair_CWP_1.columns = ['A_CWP_1', 'DATETIME']
df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
df_A_Pair_CWP_2 = pd.DataFrame(A_Pair_CWP_2_List)
df_A_Pair_CWP_2.columns = ['A_CWP_2', 'DATETIME']
df_A_Pair_CWP_2.set_index('DATETIME', inplace=True)
    
    
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:37.269221


In [33]:
# This code will use the index range (or limits) to create a list of values: feedtime range, cake wash time range,
# then selects the max value as the parameter value for that range

#FILTRATION TIME
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
A_Filt_MaxList= list()
A_Filt_MaxList[:] = []
A_feedt_Val_List = list()
A_feedt_Val_List[:] = []
A_feedt_index_List = list()
A_feedt_index_List[:] = []
A_Pair_Filt_List = list()
A_Pair_Filt_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_feedt_Val_List = list()
        A_feedt_Val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
            A_feedt = df_comb['A_t_FEED'][j]
            A_feedt_Val_List.append(A_feedt)
        A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
        A_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
    A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
except:
    pass

df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
df_A_Pair_Filt_t.columns = ['A_FILT_T', 'DATETIME']
df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))


Done 2020-01-17 13:37:37.826781


In [34]:
#MANIFOLD PRESSURE
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
A_MANFP_MaxList = list()
A_MANFP_MaxList[:] = []
A_Pair_MANFP_List =list() 
A_Pair_MANFP_List[:] = []
A_MANFP_index_List=list()      
A_MANFP_index_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_MANFP_val_List=list() 
        A_MANFP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            #Gets A manifold pressure of the cycle
            A_MANFP = df_comb['106FT02A_MANFP'][j]         
            A_MANFP_val_List.append(A_MANFP)
        A_MANFP_MaxList.append(sorted(A_MANFP_val_List)[-1])
        A_MANFP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_MANFP_List = list(zip(A_MANFP_MaxList, A_MANFP_index_List[0:]))      
                        
except:
    pass

df_A_Pair_MANF_P = pd.DataFrame(A_Pair_MANFP_List)
df_A_Pair_MANF_P.columns = ['A_MANF_P', 'DATETIME']
df_A_Pair_MANF_P.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:38.383191


In [35]:
#CWSH1
##wORKING as of 2020.01.15

keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_CWSH1_Maxlist = list()
A_t_CWSH1_Maxlist[:] = []

A_CWSH1_index_List=list()      
A_CWSH1_index_List[:] = []

A_Pair_CWSH1_List =list() 
A_Pair_CWSH1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_CWSH1_val_List=list() 
        A_t_CWSH1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_CWSH1 = df_comb['A_t_CWSH1'][j]         
            A_t_CWSH1_val_List.append(A_t_CWSH1)
        A_t_CWSH1_Maxlist.append(sorted(A_t_CWSH1_val_List)[-1])
        A_CWSH1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_CWSH1_List = list(zip(A_t_CWSH1_Maxlist, A_CWSH1_index_List[0:]))      
                        
except:
    pass
df_A_Pair_CWSH1 = pd.DataFrame(A_Pair_CWSH1_List)
df_A_Pair_CWSH1.columns = ['A_CWSH1', 'DATETIME']
df_A_Pair_CWSH1.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:38.933391


In [36]:
#CWSH2
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_CWSH2_Maxlist = list()
A_t_CWSH2_Maxlist[:] = []

A_CWSH2_index_List=list()      
A_CWSH2_index_List[:] = []

A_Pair_CWSH2_List =list() 
A_Pair_CWSH2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_CWSH2_val_List=list() 
        A_t_CWSH2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_CWSH2 = df_comb['A_t_CWSH2'][j]         
            A_t_CWSH2_val_List.append(A_t_CWSH2)
        A_t_CWSH2_Maxlist.append(sorted(A_t_CWSH2_val_List)[-1])
        A_CWSH2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_CWSH2_List = list(zip(A_t_CWSH2_Maxlist, A_CWSH2_index_List[0:]))      
                        
except:
    pass
  
df_A_Pair_CWSH2 = pd.DataFrame(A_Pair_CWSH2_List)
df_A_Pair_CWSH2.columns = ['A_CWSH2', 'DATETIME']
df_A_Pair_CWSH2.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:39.489211


In [37]:
#Pressing_1
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_Press1_Maxlist = list()
A_t_Press1_Maxlist[:] = []

A_t_Press1_index_List=list()      
A_t_Press1_index_List[:] = []

A_Pair_A_t_Press1_List =list() 
A_Pair_A_t_Press1_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_Press1_val_List=list() 
        A_t_Press1_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_Press1 = df_comb['A_t_PRESS1'][j]         
            A_t_Press1_val_List.append(A_t_Press1)
        A_t_Press1_Maxlist.append(sorted(A_t_Press1_val_List)[-1])
        A_t_Press1_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_Press1_List = list(zip(A_t_Press1_Maxlist, A_t_Press1_index_List[0:]))      
                        
except:
    pass
df_A_Pair_PRESS1_t = pd.DataFrame(A_Pair_Press1_List)
df_A_Pair_PRESS1_t.columns = ['A_PRESS1_t', 'DATETIME']
df_A_Pair_PRESS1_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:40.041298


In [38]:
#Pressing_2
##wORKING as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_t_Press2_Maxlist = list()
A_t_Press2_Maxlist[:] = []

A_t_Press2_index_List=list()      
A_t_Press2_index_List[:] = []

A_Pair_Press2_List =list() 
A_Pair_Press2_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_t_Press2_val_List=list() 
        A_t_Press2_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_t_Press2 = df_comb['A_t_PRESS2'][j]         
            A_t_Press2_val_List.append(A_t_Press2)
        A_t_Press2_Maxlist.append(sorted(A_t_Press2_val_List)[-1])
        A_t_Press2_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_Press2_List = list(zip(A_t_Press2_Maxlist, A_t_Press2_index_List[0:]))      
                        
except:
    pass
  
df_A_Pair_PRESS2_t = pd.DataFrame(A_Pair_Press2_List)
df_A_Pair_PRESS2_t.columns = ['A_PRESS2_t', 'DATETIME']
df_A_Pair_PRESS2_t.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:40.587954


In [39]:
#A_PU26A_OP
##Working as of 2020.01.15
A_PU26A_OP_Maxlist = list()
A_PU26A_OP_Maxlist[:] = []

A_PU26A_OP_index_List=list()      
A_PU26A_OP_index_List[:] = []

A_Pair_PU26A_OP_List =list() 
A_Pair_PU26A_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU26A_OP_val_List=list() 
        A_PU26A_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU26A_OP = df_comb['PU26A_OP'][j]      
            A_PU26A_OP_val_List.append(A_PU26A_OP)
        A_PU26A_OP_Maxlist.append(sorted(A_PU26A_OP_val_List)[-1])
        A_PU26A_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU26A_OP_List = list(zip(A_PU26A_OP_Maxlist, A_PU26A_OP_index_List[0:]))          
except:
    pass
df_A_Pair_PU26A_OP = pd.DataFrame(A_Pair_PU26A_OP_List)
df_A_Pair_PU26A_OP.columns = ['A_PU26A_OP', 'DATETIME']
df_A_Pair_PU26A_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:41.130163


In [40]:
#A_PU27A_OP
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU27A_OP_Maxlist = list()
A_PU27A_OP_Maxlist[:] = []

A_PU27A_OP_index_List=list()      
A_PU27A_OP_index_List[:] = []

A_Pair_PU27A_OP_List =list() 
A_Pair_PU27A_OP_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU27A_OP_val_List=list() 
        A_PU27A_OP_val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU27A_OP = df_comb['PU27A_OP'][j]      
            A_PU27A_OP_val_List.append(A_PU27A_OP)
        A_PU27A_OP_Maxlist.append(sorted(A_PU27A_OP_val_List)[-1])
        A_PU27A_OP_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU27A_OP_List = list(zip(A_PU27A_OP_Maxlist, A_PU27A_OP_index_List[0:]))          
except:
    pass
  
df_A_Pair_PU27A_OP = pd.DataFrame(A_Pair_PU27A_OP_List)
df_A_Pair_PU27A_OP.columns = ['A_PU27A_OP', 'DATETIME']
df_A_Pair_PU27A_OP.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:41.680913


In [41]:
#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU26A_CUR'])):
    if df_comb['106PU26A_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU26A_CUR'][x] = 0
#A_PU26A_CUR
##Working as of 2020.01.15
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU26A_CUR_Maxlist = list()
A_PU26A_CUR_Maxlist[:] = []

A_PU26A_CUR_index_List=list()      
A_PU26A_CUR_index_List[:] = []

A_Pair_PU26A_CUR_List =list() 
A_Pair_PU26A_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU26A_CUR_val=list() 
        A_PU26A_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU26A_CUR = df_comb['106PU26A_CUR'][j]      
            A_PU26A_CUR_val.append(A_PU26A_CUR)
        A_PU26A_CUR_Maxlist.append(sorted(A_PU26A_CUR_val)[-1])
        A_PU26A_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU26A_CUR_List = list(zip(A_PU26A_CUR_Maxlist, A_PU26A_CUR_index_List[0:]))          
except:
    pass

df_A_Pair_PU26A_CUR = pd.DataFrame(A_Pair_PU26A_CUR_List)
df_A_Pair_PU26A_CUR.columns = ['A_PU26A_CUR', 'DATETIME']
df_A_Pair_PU26A_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Done 2020-01-17 13:37:43.386747


In [42]:
#this code replaces the 'No Good Data' from 106PU26A_CUR column
for x in range(0, len(df_comb['106PU27A_CUR'])):
    if df_comb['106PU27A_CUR'][x] =='[-11059] No Good Data For Calculation':
            df_comb['106PU27A_CUR'][x] = 0
#A_PU27A_CUR
##Working as of 2020.01.15 after a million trials!!!!!!!!!! mabuak na akong utok!!!
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later

A_PU27A_CUR_Maxlist = list()
A_PU27A_CUR_Maxlist[:] = []

A_PU27A_CUR_index_List=list()      
A_PU27A_CUR_index_List[:] = []

A_Pair_PU27A_CUR_List =list() 
A_Pair_PU27A_CUR_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_PU27A_CUR_val=list() 
        A_PU27A_CUR_val[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):               
            A_PU27A_CUR = df_comb['106PU27A_CUR'][j]      
            A_PU27A_CUR_val.append(A_PU27A_CUR)
        A_PU27A_CUR_Maxlist.append(sorted(A_PU27A_CUR_val)[-1])
        A_PU27A_CUR_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    A_Pair_PU27A_CUR_List = list(zip(A_PU27A_CUR_Maxlist, A_PU27A_CUR_index_List[0:]))          
except:
    pass
  
df_A_Pair_PU27A_CUR = pd.DataFrame(A_Pair_PU27A_CUR_List)
df_A_Pair_PU27A_CUR.columns = ['A_PU27A_CUR', 'DATETIME']
df_A_Pair_PU27A_CUR.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Done 2020-01-17 13:37:44.954329


In [43]:
#A_N2 Dry
##Working as of 2020-01-15 17:26:09.736327
A_Pair_N2_Dry_List = list()
A_Pair_N2_Dry_List[:] = []
A_t_N2_Dry_indexList = list()
A_t_N2_Dry_indexList[:] = []
A_t_N2_Dry_val = list()
A_t_N2_Dry_val[:] = []
try:         
    for i in range(0, len(keyslist)):   
        A_t_N2_Dry = df_comb['A_t_DRY'][ft_cyc_lim[keyslist[i]]]
        A_t_N2_Dry_val.append(A_t_N2_Dry)
        A_t_N2_Dry_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))          
except:
    pass

df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
df_A_Pair_N2DRY.columns = ['A_N2DRY', 'DATETIME']
df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:45.012148


In [44]:
dfn = df_A_Pair_CWP_1.join(df_A_Pair_CWP_2, how='outer')
dfn1 = df_A_Pair_Filt_t.join(df_A_Pair_MANF_P, how='outer')
dfn2 = df_A_Pair_CWSH1.join(df_A_Pair_CWSH2, how='outer')
dfn3 = df_A_Pair_PRESS1_t.join(df_A_Pair_PRESS2_t, how='outer')
dfn4 = df_A_Pair_PU26A_OP.join(df_A_Pair_PU27A_OP, how='outer')
dfn5 = df_A_Pair_PU26A_CUR.join(df_A_Pair_PU27A_CUR, how='outer')
dfn6 = df_A_Pair_N2DRY.join(dfn, how='outer')
dfn7 = dfn1.join(dfn2, how='outer')
dfn8 = dfn3.join(dfn4, how='outer')
dfn9 = dfn5.join(dfn6, how='outer')
dfn10 = dfn7.join(dfn8, how='outer')
dfn11 = dfn9.join(dfn10, how='outer')
print('Done', str(datetime.datetime.now()))

Done 2020-01-17 13:37:45.102201


In [45]:
#####This block of code appends the Analysis Results from 106TH01 UF
import datetime
from dateutil.parser import parse
import pandas as pd

import openpyxl
wb1 = openpyxl.load_workbook(r'C:\Users\v.t.flores\Documents\Copy of 11.) November_ 2019.xlsx')
TH01_UF_sheet = wb1['106TH01 UF']

In [81]:
TH01_UF_sheet['S'+str(9)].value

In [82]:
TH01_UF_sheet['B'+str(9)].value

1

In [245]:
#This code finds the indexloc of the cells with day number from the sampling date column
rows_n = list()
try:
    for i in range (1, TH01_UF_sheet.max_row+1):
        if type(TH01_UF_sheet['B'+ str(i)].value) == int:
            #print(str(i)+':'+str(TH01_UF_sheet['B'+ str(i)].value))
            rows_n.append(i)
except:
    pass
#--------------------------
#This code finds the Pb concentration and the datetime
Pb_df_list = list()
Pb_df_list[:] = []
PbList = list()
PbList[:] = []
timeList = list()
timeList[:] = []

for i in range(0, len(rows_n)-1):
    for j in range(rows_n[i], rows_n[i+1]):        
        if TH01_UF_sheet['S'+str(j)].value != None and type(TH01_UF_sheet['S'+str(j)].value) == float:
            if  type(TH01_UF_sheet['C'+str(j)].value) == datetime.datetime:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = parse(str(time_val))
                newtime = str(strdate).split(" ")[1]
                if newtime == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+newtime
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+newtime
          
            elif type(TH01_UF_sheet['C'+str(j)].value) == datetime.time:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = str(time_val)
                if strdate == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+strdate
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+strdate       
        
            timeList.append(parse(newdate))
            Pb_val = TH01_UF_sheet['S'+str(j)].value
            PbList.append(Pb_val)
    #print(TH01_UF_sheet['B'+str(rows_n[i])].value)
Pb_df_list = list(zip(PbList, timeList[0:]))

#converting lists to dataframe
Pb_df = pd.DataFrame(Pb_df_list)
Pb_df.columns = ['Pb', 'DATETIME']
Pb_df.set_index('DATETIME', inplace=True)
print('done')
#--------------
#This code finds the Zn concentration and the datetime

Zn_df_list = list()
Zn_df_list[:] = []
ZnList = list()
ZnList[:] = []
ZntimeList = list()
ZntimeList[:] = []

for i in range(0, len(rows_n)-1):
    for j in range(rows_n[i], rows_n[i+1]):        
        if TH01_UF_sheet['T'+str(j)].value != None and type(TH01_UF_sheet['T'+str(j)].value) == float:
            if  type(TH01_UF_sheet['C'+str(j)].value) == datetime.datetime:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = parse(str(time_val))
                newtime = str(strdate).split(" ")[1]
                if newtime == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+newtime
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+newtime
          
            elif type(TH01_UF_sheet['C'+str(j)].value) == datetime.time:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = str(time_val)
                if strdate == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+strdate
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+strdate       
            
            timeList.append(parse(newdate))
            Zn_val = TH01_UF_sheet['T'+str(j)].value
            ZnList.append(Zn_val)
    #print(TH01_UF_sheet['B'+str(rows_n[i])].value)
Zn_df_list = list(zip(ZnList, timeList[0:]))

#converting lists to dataframe
Zn_df = pd.DataFrame(Zn_df_list)
Zn_df.columns = ['Zn', 'DATETIME']
Zn_df.set_index('DATETIME', inplace=True)

#--------------
#This code finds the Cu concentration and the datetime

Cu_df_list = list()
Cu_df_list[:] = []
CuList = list()
CuList[:] = []
CutimeList = list()
CutimeList[:] = []

for i in range(0, len(rows_n)-1):
    for j in range(rows_n[i], rows_n[i+1]):        
        if TH01_UF_sheet['U'+str(j)].value != None and type(TH01_UF_sheet['U'+str(j)].value) == float: ##check if column is updated
            if  type(TH01_UF_sheet['C'+str(j)].value) == datetime.datetime:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = parse(str(time_val))
                newtime = str(strdate).split(" ")[1]
                if newtime == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+newtime
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+newtime
          
            elif type(TH01_UF_sheet['C'+str(j)].value) == datetime.time:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = str(time_val)
                if strdate == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+strdate
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+strdate       
               
            timeList.append(parse(newdate))
            Cu_val = TH01_UF_sheet['U'+str(j)].value ##check if column and varname is updated
            CuList.append(Cu_val) ##check if list title is updated
Cu_df_list = list(zip(CuList, timeList[0:]))

#converting lists to dataframe
Cu_df = pd.DataFrame(Zn_df_list)
Cu_df.columns = ['Cu', 'DATETIME'] ##check if list title is updated
Cu_df.set_index('DATETIME', inplace=True)

#--------------
#This code finds the Mg concentration and the datetime
Mg_df_list = list()
Mg_df_list[:] = []
MgList = list()
MgList[:] = []
MgtimeList = list()
MgtimeList[:] = []

for i in range(0, len(rows_n)-1):
    for j in range(rows_n[i], rows_n[i+1]):        
        if TH01_UF_sheet['AD'+str(j)].value != None and type(TH01_UF_sheet['AD'+str(j)].value) == float: ##check if column is updated
            if  type(TH01_UF_sheet['C'+str(j)].value) == datetime.datetime:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = parse(str(time_val))
                newtime = str(strdate).split(" ")[1]
                if newtime == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+newtime
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+newtime
          
            elif type(TH01_UF_sheet['C'+str(j)].value) == datetime.time:
                time_val = TH01_UF_sheet['C'+str(j)].value           
                strdate = str(time_val)
                if strdate == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(TH01_UF_sheet['B'+str(rows_n[i+1])].value))+' '+strdate
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(TH01_UF_sheet['B'+str(rows_n[i])].value))+' '+strdate       
                     
            timeList.append(parse(newdate))
            Mg_val = TH01_UF_sheet['AD'+str(j)].value ##check if column and varname are updated
            MgList.append(Mg_val) ##check if list title is updated
Mg_df_list = list(zip(MgList, timeList[0:])) ##check if the list titles and the df list are updated

#converting lists to dataframe
Mg_df = pd.DataFrame(Mg_df_list) ##check if the element list title is updated
Mg_df.columns = ['Mg', 'DATETIME'] ##check if list title and the column name are updated
Mg_df.set_index('DATETIME', inplace=True)
#--------------

dfEl1 = Pb_df.join(Mg_df, how='outer')
dfEl2 = Cu_df.join(Zn_df, how='outer')
dfEL3 = dfEl1.join(dfEl2, how='outer')
print('Done', str(datetime.datetime.now()))


done
Done 2020-01-17 12:55:06.727257


In [248]:
#fcn for formatting the dates from range str
def datesplitter_start(x):
    return parse(x.split('to')[0])
def datesplitter_end(x):
    return parse(x.split('to')[1])
#---------
dfn11['newdt'] = dfn11.index
dfn11['startdate'] = dfn11['newdt'].apply(lambda x: datesplitter_start(x))
dfn11['enddate'] = dfn11['newdt'].apply(lambda x: datesplitter_end(x))

In [249]:
#code for date index formattter
df_anl_indexList = list()
df_anl_indexList[:] = []
df_n11_indexList = list()
df_n11_indexList[:] = []

try:
    for i in range(0, 175):
        for j in range(0, len(dfn11)):
            if dfn11['enddate'][j-1] <= dfEL3.index[i] and dfn11['enddate'][j] >= dfEL3.index[i]:
                df_n11_indexList.append(dfn11.index[j])
                df_anl_indexList.append(dfEL3.index[i]) 
except IndexError:
    print('IndexError')

Analys_indexList_Pair = list(zip(df_anl_indexList, df_n11_indexList[0:]))
dfEL4 = pd.DataFrame(Analys_indexList_Pair)

In [252]:
#code for changing the index to Datetime in range format (from date1 to date2)
df_anl_indexList = list()
df_anl_indexList[:] = []
df_n11_indexList = list()
df_n11_indexList[:] = []
try:
    for i in range(0, 175):
        for j in range(0, len(dfn11)):
            if dfn11['enddate'][j-1] <= dfEL3.index[i] and dfn11['enddate'][j] >= dfEL3.index[i]:
                df_n11_indexList.append(dfn11.index[j])
                df_anl_indexList.append(dfEL3.index[i]) 
except IndexError:
    print('IndexError')  
Analys_indexList_Pair = list(zip(df_anl_indexList, df_n11_indexList[0:]))
dfEL4 = pd.DataFrame(Analys_indexList_Pair)
dfEL4.columns = ['DATETIME', 'FT_cycletimerange']
dfEL4.set_index('DATETIME', inplace=True)

dfEL5 = dfEL3.join(dfEL4, how='outer')
dfEL5.columns = ['106TH_UF_Pb', '106TH_UF_Mg', '106TH_UF_Cu', '106TH_UF_Zn', 'DATETIME']
dfEL5.set_index('DATETIME', inplace=True)
df_Cycle_LabRes = dfn11.join(dfEL5, how='outer')

#---------------dfEL5 is the is the analysis results data indexed by daterange format. readt for merging with dfn11(FT02A data from Pi)

In [166]:
dfEL5.to_excel(r'C:\Users\v.t.flores\Documents\20200117AM_datetimerangep_i.xlsx')

In [ ]:
df_Cycle_LabRes.to_excel(r'C:\Users\v.t.flores\Documents\20200117AM_Cycle_LabRes_i.xlsx')

In [205]:
dfEL4.to_excel(r'C:\Users\v.t.flores\Documents\20200117AM_datetimerangep_dfEL4.xlsx')

In [39]:
dfn12.to_excel(r'C:\Users\v.t.flores\Documents\201911Data_Mg_Inv_20200116.xlsx')

In [6]:
# This code will use the index range (or limits) to create a list of values: feedtime range, cake wash time range,
# then selects the max value as the parameter value for that range

#FILTRATION TIME
keyslist = list(ft_cyc_lim.keys()) #creates a list of the ft_cyc_lim keys so that we can iterate later
#For loop Dictionary names 
A_Filt_MaxList= list()
A_Filt_MaxList[:] = []
A_feedt_Val_List = list()
A_feedt_Val_List[:] = []
A_feedt_index_List = list()
A_feedt_index_List[:] = []
A_Pair_Filt_List = list()
A_Pair_Filt_List[:] = []
try:         
    for i in range(0, len(keyslist)):
        A_feedt_Val_List = list()
        A_feedt_Val_List[:] = []
        for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
            A_feedt = df_comb['A_t_FEED'][j]
            A_feedt_Val_List.append(A_feedt)
        A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
        A_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
    A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
except:
    pass

df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
df_A_Pair_Filt_t.columns = ['A_FILT_T', 'DATETIME']
df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)

print('Done', str(datetime.datetime.now()))


Done 2020-01-17 13:09:33.400000


In [12]:
df_A_Pair_Filt_t.head(19)

,A_FILT_T
DATETIME,
2019-11-01 00:11:00 to 2019-11-01 00:34:00,458.0
2019-11-01 00:36:00 to 2019-11-01 00:59:00,484.0
2019-11-01 01:02:00 to 2019-11-01 01:24:00,506.0
2019-11-01 01:27:00 to 2019-11-01 01:51:00,564.0
2019-11-01 01:53:00 to 2019-11-01 02:16:00,500.0
2019-11-01 02:19:00 to 2019-11-01 02:42:00,458.0
2019-11-01 02:44:00 to 2019-11-01 03:06:00,447.0
2019-11-01 03:08:00 to 2019-11-01 03:31:00,475.0
2019-11-01 03:34:00 to 2019-11-01 03:56:00,510.0


In [13]:
range(keyslist[9], ft_cyc_lim[keyslist[9]])

range(239, 263)

In [26]:
Filtration_time_data_Nov2019 = pd.DataFrame(df_comb['A_t_FEED'])

In [28]:
Filtration_time_data_Nov2019.to_excel(r'C:\Users\v.t.flores\Documents\20200117AM_Filtration_time_data_Nov2019.xlsx')

In [46]:
FT02_Spcl = wb1['106FT02AB']

In [ ]:





Pb_df_list = list()
Pb_df_list[:] = []
PbList = list()
PbList[:] = []
timeList = list()
timeList[:] = []

for i in range(0, len(rows_prod_day)-1):
    for j in range(rows_prod_day[i], rows_prod_day[i+1]):        
        if FT02_Spcl['S'+str(j)].value != None and type(FT02_Spcl['S'+str(j)].value) == float:
            if  type(FT02_Spcl['C'+str(j)].value) == datetime.datetime:
                time_val = FT02_Spcl['C'+str(j)].value           
                strdate = parse(str(time_val))
                newtime = str(strdate).split(" ")[1]
                if newtime == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(FT02_Spcl['B'+str(rows_n[i+1])].value))+' '+newtime
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(FT02_Spcl['B'+str(rows_n[i])].value))+' '+newtime
          
            elif type(FT02_Spcl['C'+str(j)].value) == datetime.time:
                time_val = FT02_Spcl['C'+str(j)].value           
                strdate = str(time_val)
                if strdate == '03:00:00':
                    newdate = str(datetime.date(2019,11,1)-timedelta(1)+timedelta(FT02_Spcl['B'+str(rows_n[i+1])].value))+' '+strdate
                else:
                    newdate = str(datetime.date(2019,11,1)-timedelta(1) +timedelta(FT02_Spcl['B'+str(rows_n[i])].value))+' '+strdate       
        
            timeList.append(parse(newdate))
            Pb_val = FT02_Spcl['S'+str(j)].value
            PbList.append(Pb_val)
    #print(TH01_UF_sheet['B'+str(rows_n[i])].value)
Pb_df_list = list(zip(PbList, timeList[0:]))


print('done')
#--------------

In [53]:
get_column_letter(20)

'T'

In [ ]:
for i in range(0, max)
FT02_Spcl.cell.(row=i, column=j).value

In [ ]:
#This code finds the indexloc of the cells with day number from the sampling date column
rows_prod_day = list()
try:
    for i in range (1, FT02_Spcl.max_row+1):
        if type(FT02_Spcl['B'+ str(i)].value) == int:
            #print(str(i)+':'+str(TH01_UF_sheet['B'+ str(i)].value))
            rows_prod_day.append(i)
except:
    pass
#--------------------------


RosterData = {}
for i in range(2, sheet.max_row + 1):
    for j in range(2, sheet.max_column + 1):
        
        Smpl_t = sheet.cell(row=i, column=4).value.split(',')[0].upper()
        date = sheet.cell(row=i, column=13).value
        shift = sheet.cell(row=i, column=12).value
        
        if  nameP is not None:
            if date is not None:
                if shift is not None:            
                    RosterData.setdefault(nameP, {})
                    RosterData[nameP].setdefault(date, shift)
                    

In [ ]:
ft02A_smpl_t_index = list()
ft02A_smpl_t_index[:] = []
for i in range(0, len(rows_prod_day)):
    print(FT02_Spcl['B'+ str(rows_prod_day[i])].value)
    for j in (rows_prod_day[i], rows_prod_day[i]):
        if FT02_Spcl['C'+ str(j)].value =='106FT02A':
            ft02A_smpl_t_index.append(j)
            print(ft02A_smpl_t_index.append(j))

In [157]:
rows_prod_day[-1]

325

In [141]:
for k in range(0, len(rows_prod_day)):
    if rows_prod[k]
        for m in range(rows_prod_day[k], rows_prod_day[k+1]):
            if FT02_Spcl['C'+ str(m)].value =='106FT02A':
                print(FT02_Spcl['C'+ str(m)].value)
                #a = str(FT02_Spcl['D'+ str(m)].value).split(':')[0]
                #b = str(FT02_Spcl['D'+ str(m)].value).split(':')[1]
                print(datetime.date(2019,11,1)-datetime.timedelta(1)+datetime.timedelta(FT02_Spcl['B'+ str(rows_prod_day[k])].value))

106FT02A
2019-11-02
106FT02A
2019-11-02
106FT02A
2019-11-02
106FT02A
2019-11-03
106FT02A
2019-11-03
106FT02A
2019-11-03
106FT02A
2019-11-03
106FT02A
2019-11-04
106FT02A
2019-11-04
106FT02A
2019-11-04
106FT02A
2019-11-05
106FT02A
2019-11-05
106FT02A
2019-11-05
106FT02A
2019-11-06
106FT02A
2019-11-06
106FT02A
2019-11-06
106FT02A
2019-11-07
106FT02A
2019-11-07
106FT02A
2019-11-07
106FT02A
2019-11-08
106FT02A
2019-11-08
106FT02A
2019-11-08
106FT02A
2019-11-08
106FT02A
2019-11-09
106FT02A
2019-11-09
106FT02A
2019-11-09
106FT02A
2019-11-10
106FT02A
2019-11-10
106FT02A
2019-11-11
106FT02A
2019-11-11
106FT02A
2019-11-11
106FT02A
2019-11-11
106FT02A
2019-11-11
106FT02A
2019-11-12
106FT02A
2019-11-12
106FT02A
2019-11-12
106FT02A
2019-11-13
106FT02A
2019-11-13
106FT02A
2019-11-13
106FT02A
2019-11-14
106FT02A
2019-11-14
106FT02A
2019-11-14
106FT02A
2019-11-15
106FT02A
2019-11-15
106FT02A
2019-11-15
106FT02A
2019-11-16
106FT02A
2019-11-16
106FT02A
2019-11-17
106FT02A
2019-11-17
106FT02A
2019-11-17


In [117]:
str(datetime.date(2019,11,1)+datetime.timedelta(1))

'2019-11-02'

In [115]:
FT02_Spcl['D'+ str(10)].value

datetime.time(5, 46)

In [143]:
FT02_Spcl['B'+ str(325)].value

30

In [153]:
for i in range(1, FT02_Spcl.max_row + 1):
    if FT02_Spcl['C'+ str(i)].value == '106FT02A' or FT02_Spcl['C'+ str(i)].value == '106FT02B':
        a = i 

In [168]:
a

333

In [167]:
for k in range(0, len(rows_prod_day)):
    for m in range(rows_prod_day[k], rows_prod_day[k+1]):
        if FT02_Spcl['C'+ str(m)].value =='106FT02A':
            #print(k,m)

SyntaxError: unexpected EOF while parsing (<ipython-input-167-e7afc1a14549>, line 4)

In [234]:
for i in range(1, FT02_Spcl.max_row + 1):
    if FT02_Spcl['C'+ str(i)].value == '106FT02A' or FT02_Spcl['C'+ str(i)].value == '106FT02B':
        a = i


for k in range(0, len(rows_prod_day)): # index numbers for the days of the month of november
    if rows_prod_day[k] != rows_prod_day[-1]: # checking if the cell index is the last in the indexlist rows_prod_day
        for m in range(rows_prod_day[k], rows_prod_day[k+1]): # loop for range between each day in november
            if FT02_Spcl['C'+str(rows_prod_day[k])].value == '106FT02A': # check if this is a product from A
                a = FT02_Spcl['D'+ str(m)].value
                print(a)
            elif FT02_Spcl['C'+str(rows_prod_day[k])].value == '106FT02B':# check if this is a product from B
                b = FT02_Spcl['D'+ str(m)].value    
                print(b)

    elif rows_prod_day[k] == rows_prod_day[-1]: #for the case where the cell is the last in the indexlist of the rows_prod_day
        for s in range(rows_prod_day[k], 333): #fix the 333 later!!!
            if type(FT02_Spcl['D'+str(s)].value) == datetime.time:
                if FT02_Spcl['C'+str(rows_prod_day[k])].value == '106FT02A': # check if this is a product from A
                    c = FT02_Spcl['D'+ str(s)].value
                    print(c)
                elif FT02_Spcl['C'+str(rows_prod_day[k])].value == '106FT02B':# check if this is a product from B
                    d = FT02_Spcl['D'+ str(s)].value    
                    print(d)
                
                #print(datetime.date(2019,11,1)-datetime.timedelta(1)+datetime.timedelta(FT02_Spcl['B'+ str(rows_prod_day[k])].value))
                #print(FT02_Spcl['D'+ str(s)].value)

            

01:18:00
01:03:00
None
04:57:00
23:48:00
None
09:33:00
10:17:00
None
13:05:00
13:00:00
None
16:47:00
16:42:00
None


In [ ]:
for k in range(0, len(rows_prod_day)):
    if rows_prod[k]
        for m in range(rows_prod_day[k], rows_prod_day[k+1]):
            if FT02_Spcl['C'+ str(m)].value =='106FT02A':
                print(FT02_Spcl['C'+ str(m)].value)
                #a = str(FT02_Spcl['D'+ str(m)].value).split(':')[0]
                #b = str(FT02_Spcl['D'+ str(m)].value).split(':')[1]
                print(datetime.date(2019,11,1)-datetime.timedelta(1)+datetime.timedelta(FT02_Spcl['B'+ str(rows_prod_day[k])].value))

In [170]:
print(datetime.date(2019,11,1)-datetime.timedelta(1)+datetime.timedelta(FT02_Spcl['B'+ str(rows_prod_day[29])].value))

2019-11-30
